In [ ]:
import pandas as pd
import numpy as np
import os
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdmetrics.reports.single_table import QualityReport
from sdmetrics.reports.single_table import DiagnosticReport
from table_evaluator import TableEvaluator

In [ ]:
#creating a combination of real data

# real_data = pd.read_csv('Datasets/Preprocessed_Datasets/doS_attacks.csv')
# brute = pd.read_csv('Datasets/Preprocessed_Datasets/bruteforce_attacks.csv')
# goldenEye = real_data[real_data.Label=='DoS attacks-GoldenEye']
# slowloris = real_data[real_data.Label=='DoS attacks-Slowloris']
# hulk = real_data[real_data.Label=='DoS attacks-Hulk']
# hulk = hulk.iloc[:300000, :]
# slowHTTPtest = real_data[real_data.Label=='DoS attacks-SlowHTTPTest']
# loicHTTp = real_data[real_data.Label=='DDoS attacks-LOIC-HTTP']
# loicHTTp = loicHTTp.iloc[:300000, :]
# hoic = real_data[real_data.Label=='DDOS attack-HOIC']
# hoic = hoic.iloc[:300000, :]
# bot = pd.read_csv('Datasets/Preprocessed_Datasets/bot_attacks.csv')
# infilteration = pd.read_csv('Datasets/Preprocessed_Datasets/infilteration_attacks.csv')
# benign = pd.read_csv('Datasets/Preprocessed_Datasets/benign.csv')
# benign = benign.iloc[:300000, :]
# ftp = brute[brute.Label=='FTP-BruteForce']
# ssh = brute[brute.Label=='SSH-Bruteforce']

# real_all_in_one = pd.concat([goldenEye, slowloris, hulk, slowHTTPtest, loicHTTp, hoic, bot, infilteration, benign, ftp, ssh], ignore_index=True)

In [ ]:
# reading all csv files in the directory

folder_path = 'RTVAE_Results'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    dfs.append(pd.read_csv(file_path))
    
synthetic_all_in_one = pd.concat(dfs, ignore_index=True)

In [ ]:
#savign real and synthetic data
# real_all_in_one.to_csv('TVAE_Results/real_all_in_one.csv', index=False)
synthetic_all_in_one.to_csv('RTVAE_Results/RTVAE_synthetic_all.csv', index=False)

In [ ]:
#ensuring that everthing went well

synthetic_data = synthetic_all_in_one 
print(synthetic_data.shape)
synthetic_data.dropna(inplace=True)
print(synthetic_data.shape)

In [ ]:
#Loading real and sytnehtic data for evaluation

real_data = pd.read_csv('TVAE_Results/real_all_in_one.csv')
synthetic_data = pd.read_csv('TabFairGAN_Results/TabFairGAN_synthetic_all.csv')

In [ ]:
def get_data_info(df):
    """Crates the categorical columns, continuous columns, and metadata of a dataframe.

    Args:
        df (pandas.Dataframe): The input dataframe containing continuous and categorical values.

    Returns:
        list: the list of categorical column names. Specifically, columns with only 4 uniques values
        list: The list of continuous column names.
        metadata: The metadata of the dataframe. for more informatin visit https://docs.sdv.dev/sdv/reference/metadata-spec/single-table-metadata-json
    """
    #createing 
    categorical_columns = ['Label']
    continuous_columns = []
    for i in df.columns:
        if i not in categorical_columns:
            continuous_columns.append(i)
    
    #creating metadat
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(df)
    
    for column in categorical_columns:
        metadata.update_column(
            column_name = column,
            sdtype = 'categorical'
        )
    
    for column in continuous_columns:
        metadata.update_column(
            column_name = column,
            sdtype = 'numerical'  
        )
    # validating metadata
    metadata.validate()
    metadata.validate_data(data=real_data)
    
    return categorical_columns, continuous_columns, metadata


categorical_columns, continuous_columns, metadata = get_data_info(real_data)

In [ ]:
# evaluating synthetic data with table_evaluator cumulative sum per features and distribution
table_evaluator = TableEvaluator(real_data, synthetic_data, cat_cols = categorical_columns)
table_evaluator.visual_evaluation()

In [ ]:
#saving and visualizing column pair trend and column shapes
metadata = metadata.to_dict()
my_report = QualityReport()
my_report.generate(real_data, synthetic_data, metadata)
my_report.save(filepath='RTVAE_Results/quality.pkl')
my_report.get_visualization(property_name='Column Pair Trends')

In [ ]:
#saving and visualiztation data validity
my_report = DiagnosticReport()
my_report.generate(real_data, synthetic_data, metadata)
my_report.save(filepath='RTVAE_Results/diagnostic.pkl')
my_report.get_visualization('Data Validity')